In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.linear_model import Lasso
from sklearn.feature_selection import RFE
from scipy.stats import shapiro, ranksums, ttest_ind, levene

In [ ]:
data_dir = './P10/'
out_dir = './P10/plan2/'

In [ ]:
cl = pd.read_csv(data_dir+'train_cl.csv')

In [ ]:
import numpy as np
from collections import Counter
from scipy.stats import chi2_contingency
from scipy.stats import fisher_exact
from scipy.stats import pearsonr, ttest_ind, levene


In [ ]:
cl_data0 = cl[cl['label'] == 0]
cl_data1 = cl[cl['label'] == 1]

In [ ]:
from statsmodels.stats.multitest import multipletests
data0 = cl_data0
data1 = cl_data1
cl_ST = []
p_values = [] 
col_names = []  
for colName in cl.columns[2:]: 
    current_p = None  
    if colName == 'Sex'or colName == 'HBP'or colName == 'DM'or colName == 'smoke'or colName == 'drink'or colName == 'Re': 
        data0_col = Counter(data0[colName])
        data1_col = Counter(data1[colName])
        
        a = data0_col[0]
        b = data1_col[0]
        c = data0_col[1]
        d = data1_col[1]
        cross_table = np.array([[a,b],
                                [c,d]])
        n = a + b + c + d
        Ta = ((a + b)*(a + c))/n
        Tb = ((a + b)*(b + d))/n
        Tc = ((a + c)*(c + d))/n
        Td = ((c + d)*(b + d))/n
        T = min(Ta, Tb, Tc, Td)
        if((T >= 5) & (n >= 40)):
            kf = chi2_contingency(cross_table)
        elif((T < 5) & (T >= 1) & (n >= 40)):
            kf = chi2_contingency(cross_table,correction=True)
        else:
            kf = fisher_exact(cross_table, alternative='greater')
#         re.loc[0,colName] = "%.10f"%kf[1]
        if kf[1] < 0.05:
            cl_ST.append(colName)
        current_p = kf[1]
        print(colName,":\n\tP-value = %.4f"%kf[1])
    else:
        if shapiro(data0[colName])[1] > 0.05 and shapiro(data1[colName])[1] > 0.05:
            if levene(data0[colName], data1[colName])[1] > 0.05:
                P = ttest_ind(data0[colName],data1[colName])[1]
                current_p = P
                print(colName,':\n\tP-value =  %.4f'%P)
                if P < 0.05:
                    cl_ST.append(colName)       
                             
            else:
                P = ttest_ind(data0[colName],data1[colName],equal_var = False)[1]
                current_p = P
                print(colName,':\n\tP-value =  %.4f'%P)
                if P < 0.05:
                    cl_ST.append(colName)    
        else:
            P = ranksums(data0[colName], data1[colName])[1]
            current_p = P
            print(colName,':\n\tP-value =  %.4f'%P)
            if P < 0.05:
                cl_ST.append(colName)  
    p_values.append(current_p)
    col_names.append(colName)
    
rejections, pvals_corrected, _, _ = multipletests(p_values, alpha=0.05, method='fdr_bh')

for colName, reject, pval in zip(col_names, rejections, pvals_corrected):
    print(colName, ": Adjusted P-value = %.4f" % pval)
    if reject:
        cl_ST.append(colName)


In [ ]:
if 'AD' not in cl_ST:cl_ST = ['AD']+cl_ST
if 'label' not in cl_ST:cl_ST = ['label']+cl_ST

In [ ]:
cl = cl[cl_ST]

In [ ]:
ra = pd.read_csv(data_dir+'train_ra.csv')

In [ ]:
ra_data0 = ra[ra['label'] == 0]
ra_data1 = ra[ra['label'] == 1]

In [ ]:
data0 = ra_data0
data1 = ra_data1
ra_ST = []
p_values = []  
for colName in ra.columns[2:]: 
    p_value = None
    if shapiro(data0[colName])[1] > 0.05 and shapiro(data1[colName])[1] > 0.05:
        if levene(data0[colName], data1[colName])[1] > 0.05:
            p_value = ttest_ind(data0[colName], data1[colName])[1]                    
        else:
            p_value = ttest_ind(data0[colName], data1[colName], equal_var=False)[1]
    else:
        p_value = ranksums(data0[colName], data1[colName])[1]

    p_values.append(p_value)
rejections, pvals_corrected, _, _ = multipletests(p_values, alpha=0.05, method='fdr_bh')

for colName, reject in zip(ra.columns[2:], rejections):
    if reject:
        ra_ST.append(colName)


In [ ]:
if 'AD' not in ra_ST:ra_ST = ['AD']+ra_ST
if 'label' not in ra_ST:ra_ST = ['label']+ra_ST

In [ ]:
ra = ra[ra_ST]

In [ ]:
me = pd.read_csv(data_dir+'train_me.csv')

In [ ]:
me_data0 = me[me['label'] == 0]
me_data1 = me[me['label'] == 1]

In [ ]:
data0 = me_data0
data1 = me_data1
me_ST = []
p_values = []

for colName in me.columns[2:]: 
    p_value = None
    if shapiro(data0[colName])[1] > 0.05 and shapiro(data1[colName])[1] > 0.05:
        if levene(data0[colName], data1[colName])[1] > 0.05:
            p_value = ttest_ind(data0[colName], data1[colName])[1]                    
        else:
            p_value = ttest_ind(data0[colName], data1[colName], equal_var=False)[1]
    else:
        p_value = ranksums(data0[colName], data1[colName])[1]

    p_values.append(p_value)

rejections, pvals_corrected, _, _ = multipletests(p_values, alpha=0.05, method='fdr_bh')

for colName, reject in zip(me.columns[2:], rejections):
    if reject:
        me_ST.append(colName)


In [ ]:
if 'AD' not in me_ST:me_ST = ['AD']+me_ST
if 'label' not in me_ST:me_ST = ['label']+me_ST

In [ ]:
me = me[me_ST]

In [ ]:
scaler = StandardScaler()
cols_to_scale = cl.columns[2:]
scaled_data = scaler.fit_transform(cl[cols_to_scale])
cl_zscore = pd.concat([cl.iloc[:, :2], pd.DataFrame(scaled_data, columns=cols_to_scale)], axis=1)

In [ ]:
X = cl_zscore[cl_zscore.columns[2:]]
y = cl_zscore['label']
feature_names =  X.columns

In [ ]:
print(X.shape)

In [ ]:
# lasso for further feature selection
alphas = np.logspace(-18,1,50)
model_lassoCV_cl = LassoCV(alphas = alphas, cv = 10, max_iter = 10000000).fit(X,y)
coefs_cl = model_lassoCV_cl.path(X,y,alphas = alphas, max_iter = 10000000)[1].T

MSEs_cl = model_lassoCV_cl.mse_path_
MSEs_mean_cl = np.apply_along_axis(np.mean,1,MSEs_cl)
MSEs_std_cl = np.apply_along_axis(np.std,1,MSEs_cl)

In [ ]:
print(model_lassoCV_cl.alpha_)
coef = pd.Series(model_lassoCV_cl.coef_,index = X.columns)
print('Lasso picked ' + str(sum(coef != 0))+ ' variables and eliminated the other ' + str(sum(coef == 0)))

In [ ]:
cl_col_lasso = coef[coef != 0].index
cl_col_lasso = cl_col_lasso.tolist()

if 'AD' not in cl_col_lasso:cl_col_lasso = ['AD']+cl_col_lasso
if 'label' not in cl_col_lasso:cl_col_lasso = ['label']+cl_col_lasso

cl_lasso = cl[cl_col_lasso]
print(cl_lasso.shape)    

In [ ]:
if len(cl_col_lasso)>8:
    lasso = Lasso(alpha=model_lassoCV_cl.alpha_, max_iter=10000000)
    selector = RFE(lasso, n_features_to_select=6, step=1)
    selector = selector.fit(X, y)

    selected_features = feature_names[selector.support_]
    print("Selected features:", selected_features)
    selected_features=selected_features.tolist()
    if 'AD' not in selected_features:selected_features = ['AD']+selected_features
    if 'label' not in selected_features:selected_features = ['label']+selected_features
    cl_RFE = cl_lasso[selected_features]
else:
    cl_RFE = cl_lasso

In [ ]:
scaler = StandardScaler()
cols_to_scale = ra.columns[2:]
scaled_data = scaler.fit_transform(ra[cols_to_scale])
ra_zscore = pd.concat([ra.iloc[:, :2], pd.DataFrame(scaled_data, columns=cols_to_scale)], axis=1)

In [ ]:
X = ra_zscore[ra_zscore.columns[2:]]
y = ra_zscore['label']
feature_names =  X.columns
print(X.shape)

# lasso for further feature selection
alphas = np.logspace(-2,1,50)
model_lassoCV_ra = LassoCV(alphas = alphas, cv = 10, max_iter = 10000000).fit(X,y)
coefs_ra = model_lassoCV_ra.path(X,y,alphas = alphas, max_iter = 10000000)[1].T

MSEs_ra = model_lassoCV_ra.mse_path_
MSEs_mean_ra = np.apply_along_axis(np.mean,1,MSEs_ra)
MSEs_std_ra = np.apply_along_axis(np.std,1,MSEs_ra)


In [ ]:
print(model_lassoCV_ra.alpha_)
coef = pd.Series(model_lassoCV_ra.coef_,index = X.columns)
print('Lasso picked ' + str(sum(coef != 0))+ ' variables and eliminated the other ' + str(sum(coef == 0)))

ra_col_lasso = coef[coef != 0].index
ra_col_lasso = ra_col_lasso.tolist()

if 'AD' not in ra_col_lasso:ra_col_lasso = ['AD']+ra_col_lasso
if 'label' not in ra_col_lasso:ra_col_lasso = ['label']+ra_col_lasso

ra_lasso = ra[ra_col_lasso]
print(ra_lasso.shape)    

In [ ]:
if len(ra_col_lasso)>8:
    lasso = Lasso(alpha=model_lassoCV_ra.alpha_, max_iter=100000)
    selector = RFE(lasso, n_features_to_select=6, step=1,)
    selector = selector.fit(X, y)

    selected_features = feature_names[selector.support_]
    print("Selected features:", selected_features)
    selected_features = selected_features.tolist()
    if 'AD' not in selected_features:selected_features = ['AD']+selected_features
    if 'label' not in selected_features:selected_features = ['label']+selected_features
    ra_RFE = ra_lasso[selected_features]
else:
    ra_RFE = ra_lasso

In [ ]:
scaler = StandardScaler()

cols_to_scale = me.columns[2:]
scaled_data = scaler.fit_transform(me[cols_to_scale])
me_zscore = pd.concat([me.iloc[:, :2], pd.DataFrame(scaled_data, columns=cols_to_scale)], axis=1)

In [ ]:
X = me_zscore[me_zscore.columns[2:]]
y = me_zscore['label']
feature_names =  X.columns
print(X.shape)

# lasso for further feature selection
alphas = np.logspace(-2,1,50)
model_lassoCV_me = LassoCV(alphas = alphas, cv = 10, max_iter = 10000000).fit(X,y)
coefs_me = model_lassoCV_me.path(X,y,alphas = alphas, max_iter = 10000000)[1].T

MSEs_me = model_lassoCV_me.mse_path_
MSEs_mean_me = np.apply_along_axis(np.mean,1,MSEs_me)
MSEs_std_me = np.apply_along_axis(np.std,1,MSEs_me)


In [ ]:
print(model_lassoCV_me.alpha_)
coef = pd.Series(model_lassoCV_me.coef_,index = X.columns)
print('Lasso picked ' + str(sum(coef != 0))+ ' variables and eliminated the other ' + str(sum(coef == 0)))

me_col_lasso = coef[coef != 0].index
me_col_lasso = me_col_lasso.tolist()

if 'AD' not in me_col_lasso:me_col_lasso = ['AD']+me_col_lasso
if 'label' not in me_col_lasso:me_col_lasso = ['label']+me_col_lasso

me_lasso = me[me_col_lasso]
print(me_lasso.shape)   


In [ ]:
# me_lasso.to_csv(out_dir+'train_me_withoutRFE.csv', index=False)

In [ ]:
if len(me_col_lasso)>8:
    lasso = Lasso(alpha=model_lassoCV_me.alpha_, max_iter=100000)
    selector = RFE(lasso, n_features_to_select=6, step=1)
    selector = selector.fit(X, y)

    selected_features = feature_names[selector.support_]
    print("Selected features:", selected_features)
    selected_features = selected_features.tolist()
    
    if 'AD' not in selected_features:selected_features = ['AD']+selected_features
    if 'label' not in selected_features:selected_features = ['label']+selected_features
    me_RFE = me_lasso[selected_features]
else:
    me_RFE = me_lasso
    

In [ ]:
ClRa = pd.merge(ra, cl, on='AD')
if ClRa['label_x'].equals(ClRa['label_y']):
    print('The two columns are identical.')
    ClRa = ClRa.drop(columns=['label_y'])
    ClRa = ClRa.rename(columns={'label_x': 'label'})
else:
    print('The two columns are different.')

scaler = StandardScaler()

cols_to_scale = ClRa.columns[2:]

scaled_data = scaler.fit_transform(ClRa[cols_to_scale])

ClRa_zscore = pd.concat([ClRa.iloc[:, :2], pd.DataFrame(scaled_data, columns=cols_to_scale)], axis=1)


In [ ]:
X = ClRa_zscore[ClRa_zscore.columns[2:]]
y = ClRa_zscore['label']
feature_names =  X.columns

# lasso for further feature selection
alphas = np.logspace(-2,1,50)
model_lassoCV_ClRa = LassoCV(alphas = alphas, cv = 10, max_iter = 100000).fit(X,y)
coefs_ClRa = model_lassoCV_ClRa.path(X,y,alphas = alphas, max_iter = 100000)[1].T

MSEs_ClRa = model_lassoCV_ClRa.mse_path_
MSEs_mean_ClRa = np.apply_along_axis(np.mean,1,MSEs_ClRa)
MSEs_std_ClRa = np.apply_along_axis(np.std,1,MSEs_ClRa)

In [ ]:
print(model_lassoCV_ClRa.alpha_)
coef = pd.Series(model_lassoCV_ClRa.coef_,index = X.columns)
print('Lasso picked ' + str(sum(coef != 0))+ ' variables and eliminated the other ' + str(sum(coef == 0)))

ClRa_col_lasso = coef[coef != 0].index
ClRa_col_lasso = ClRa_col_lasso.tolist()

if 'AD' not in ClRa_col_lasso:ClRa_col_lasso = ['AD']+ClRa_col_lasso
if 'label' not in ClRa_col_lasso:ClRa_col_lasso = ['label']+ClRa_col_lasso
    
print(ClRa_col_lasso)
ClRa_lasso = ClRa[ClRa_col_lasso]
print(ClRa_lasso.shape)    

In [ ]:
if len(ClRa_col_lasso)>8:
    lasso = Lasso(alpha=model_lassoCV_ClRa.alpha_, max_iter=100000)
    selector = RFE(lasso, n_features_to_select=6, step=1)
    selector = selector.fit(X, y)

    selected_features = feature_names[selector.support_]
    print("Selected features:", selected_features)
    selected_features = selected_features.tolist()
    
    if 'AD' not in selected_features:selected_features = ['AD']+selected_features
    if 'label' not in selected_features:selected_features = ['label']+selected_features
    ClRa_RFE = ClRa_lasso[selected_features]
else:
    ClRa_RFE = ClRa_lasso
    

In [ ]:
ClMe = pd.merge(cl, me, on='AD')
if ClMe['label_x'].equals(ClMe['label_y']):
    print('The two columns are identical.')
    ClMe = ClMe.drop(columns=['label_y'])
    ClMe = ClMe.rename(columns={'label_x': 'label'})
else:
    print('The two columns are different.')

scaler = StandardScaler()

cols_to_scale = ClMe.columns[2:]

scaled_data = scaler.fit_transform(ClMe[cols_to_scale])

ClMe_zscore = pd.concat([ClMe.iloc[:, :2], pd.DataFrame(scaled_data, columns=cols_to_scale)], axis=1)

In [ ]:
X = ClMe_zscore[ClMe_zscore.columns[2:]]
y = ClMe_zscore['label']
feature_names = X.columns

# lasso for further feature selection
alphas = np.logspace(-3,1,50)
model_lassoCV_ClMe = LassoCV(alphas = alphas, cv = 10, max_iter = 100000).fit(X,y)
coefs_ClMe = model_lassoCV_ClMe.path(X,y,alphas = alphas, max_iter = 100000)[1].T

MSEs_ClMe = model_lassoCV_ClMe.mse_path_
MSEs_mean_ClMe = np.apply_along_axis(np.mean,1,MSEs_ClMe)
MSEs_std_ClMe = np.apply_along_axis(np.std,1,MSEs_ClMe)

In [ ]:
print(model_lassoCV_ClMe.alpha_)
coef = pd.Series(model_lassoCV_ClMe.coef_,index = X.columns)
print('Lasso picked ' + str(sum(coef != 0))+ ' variables and eliminated the other ' + str(sum(coef == 0)))

ClMe_col_lasso = coef[coef != 0].index
ClMe_col_lasso = ClMe_col_lasso.tolist()

if 'AD' not in ClMe_col_lasso:ClMe_col_lasso = ['AD']+ClMe_col_lasso
if 'label' not in ClMe_col_lasso:ClMe_col_lasso = ['label']+ClMe_col_lasso
    
print(ClMe_col_lasso)
ClMe_lasso = ClMe[ClMe_col_lasso]
print(ClMe_lasso.shape)    


In [ ]:
if len(ClMe_col_lasso)>8:
    lasso = Lasso(alpha=model_lassoCV_ClMe.alpha_, max_iter=100000)
    selector = RFE(lasso, n_features_to_select=6, step=1)
    selector = selector.fit(X, y)

    selected_features = feature_names[selector.support_]
    print("Selected features:", selected_features)
    selected_features = selected_features.tolist()
    
    if 'AD' not in selected_features:selected_features = ['AD']+selected_features
    if 'label' not in selected_features:selected_features = ['label']+selected_features
    ClMe_RFE = ClMe_lasso[selected_features]
else:
    ClMe_RFE = ClMe_lasso
    
ClMe_RFE.to_csv(out_dir+'train_clme_RFE.csv', index=False)

In [ ]:
plt.rcParams['font.family'] = 'Times New Roman'
fig, axs = plt.subplots(2, 4, figsize=(24, 9))

axs[0, 0].errorbar(model_lassoCV_ClRa.alphas_, MSEs_mean_ClRa, yerr = MSEs_std_ClRa,fmt = 'o', ms = 3, mfc = 'r', 
                   mec = 'r', ecolor = 'lightblue',elinewidth = 2, capsize = 4, capthick = 1)
axs[0, 0].set_xlabel('log(lambda)',fontsize=29)
axs[0, 0].set_ylabel('MSE', fontsize=29)
axs[0, 0].tick_params(axis='x', labelsize=17, width=2, length=4)
axs[0, 0].tick_params(axis='y', labelsize=17, width=2, length=4)
axs[0, 0].semilogx()
axs[0, 0].axvline(model_lassoCV_ClRa.alpha_,color = 'black',ls = '--')
axs[0, 0].set_title("ClRa", fontsize=30)

axs[0, 1].errorbar(model_lassoCV_ClMe.alphas_, MSEs_mean_ClMe, yerr = MSEs_std_ClMe,fmt = 'o', ms = 3, mfc = 'r', 
                   mec = 'r', ecolor = 'lightblue',elinewidth = 2, capsize = 4, capthick = 1)
axs[0, 1].set_xlabel('log(lambda)',fontsize=29)
axs[0, 1].set_ylabel('MSE', fontsize=29)
axs[0, 1].tick_params(axis='x', labelsize=17, width=2, length=4)
axs[0, 1].tick_params(axis='y', labelsize=17, width=2, length=4)
axs[0, 1].semilogx()
axs[0, 1].axvline(model_lassoCV_ClMe.alpha_,color = 'black',ls = '--')
axs[0, 1].set_title("ClMe", fontsize=30)

axs[0, 2].errorbar(model_lassoCV_RaMe.alphas_, MSEs_mean_RaMe, yerr = MSEs_std_RaMe,fmt = 'o', ms = 3, mfc = 'r', 
                   mec = 'r', ecolor = 'lightblue',elinewidth = 2, capsize = 4, capthick = 1)
axs[0, 2].set_xlabel('log(lambda)',fontsize=29)
axs[0, 2].set_ylabel('MSE', fontsize=29)
axs[0, 2].tick_params(axis='x', labelsize=17, width=2, length=4)
axs[0, 2].tick_params(axis='y', labelsize=17, width=2, length=4)
axs[0, 2].semilogx()
axs[0, 2].axvline(model_lassoCV_RaMe.alpha_,color = 'black',ls = '--')
axs[0, 2].set_title("RaMe", fontsize=30)

axs[0, 3].errorbar(model_lassoCV_ClRaMe.alphas_, MSEs_mean_ClRaMe, yerr = MSEs_std_ClRaMe,fmt = 'o', ms = 3, mfc = 'r', 
                   mec = 'r', ecolor = 'lightblue',elinewidth = 2, capsize = 4, capthick = 1)
axs[0, 3].set_xlabel('log(lambda)',fontsize=29)
axs[0, 3].set_ylabel('MSE', fontsize=29)
axs[0, 3].tick_params(axis='x', labelsize=17, width=2, length=4)
axs[0, 3].tick_params(axis='y', labelsize=17, width=2, length=4)
axs[0, 3].semilogx()
axs[0, 3].axvline(model_lassoCV_ClRaMe.alpha_,color = 'black',ls = '--')
axs[0, 3].set_title("ClRaMe", fontsize=30)

axs[1, 0].set_xlabel('log(lambda)',fontsize=29)
axs[1, 0].set_ylabel('Coefficient',fontsize=29)
axs[1, 0].tick_params(axis='x', labelsize=17, width=2, length=4)
axs[1, 0].tick_params(axis='y', labelsize=17, width=2, length=4)
axs[1, 0].semilogx(model_lassoCV_ClRa.alphas_,coefs_ClRa,'-')
axs[1, 0].axvline(model_lassoCV_ClRa.alpha_,color = 'black',ls = '--')
axs[1, 0].set_title("ClRa", fontsize=30)

axs[1, 1].set_xlabel('log(lambda)',fontsize=29)
axs[1, 1].set_ylabel('Coefficient',fontsize=29)
axs[1, 1].tick_params(axis='x', labelsize=17, width=2, length=4)
axs[1, 1].tick_params(axis='y', labelsize=17, width=2, length=4)
axs[1, 1].semilogx(model_lassoCV_ClMe.alphas_,coefs_ClMe,'-')
axs[1, 1].axvline(model_lassoCV_ClMe.alpha_,color = 'black',ls = '--')
axs[1, 1].set_title("ClMe", fontsize=30)

axs[1, 2].set_xlabel('log(lambda)',fontsize=29)
axs[1, 2].set_ylabel('Coefficient',fontsize=29)
axs[1, 2].tick_params(axis='x', labelsize=17, width=2, length=4)
axs[1, 2].tick_params(axis='y', labelsize=17, width=2, length=4)
axs[1, 2].semilogx(model_lassoCV_RaMe.alphas_,coefs_RaMe,'-')
axs[1, 2].axvline(model_lassoCV_RaMe.alpha_,color = 'black',ls = '--')
axs[1, 2].set_title("RaMe", fontsize=30)

axs[1, 3].set_xlabel('log(lambda)',fontsize=29)
axs[1, 3].set_ylabel('Coefficient',fontsize=29)
axs[1, 3].tick_params(axis='x', labelsize=17, width=2, length=4)
axs[1, 3].tick_params(axis='y', labelsize=17, width=2, length=4)
axs[1, 3].semilogx(model_lassoCV_ClRaMe.alphas_,coefs_ClRaMe,'-')
axs[1, 3].axvline(model_lassoCV_ClRaMe.alpha_,color = 'black',ls = '--')
axs[1, 3].set_title("ClRaMe", fontsize=30)

axs[0, 0].annotate("a.", (-0.22, 1.07), xycoords='axes fraction', fontsize=30, fontweight='bold', va='top')
axs[0, 1].annotate("b.", (-0.22, 1.07), xycoords='axes fraction', fontsize=30, fontweight='bold', va='top')
axs[0, 2].annotate("c.", (-0.22, 1.07), xycoords='axes fraction', fontsize=30, fontweight='bold', va='top')
axs[0, 3].annotate("d.", (-0.22, 1.07), xycoords='axes fraction', fontsize=30, fontweight='bold', va='top')
axs[1, 0].annotate("e.", (-0.22, 1.07), xycoords='axes fraction', fontsize=30, fontweight='bold', va='top')
axs[1, 1].annotate("f.", (-0.22, 1.07), xycoords='axes fraction', fontsize=30, fontweight='bold', va='top')
axs[1, 2].annotate("g.", (-0.22, 1.07), xycoords='axes fraction', fontsize=30, fontweight='bold', va='top')
axs[1, 3].annotate("h.", (-0.22, 1.07), xycoords='axes fraction', fontsize=30, fontweight='bold', va='top')

fig.subplots_adjust(hspace=0.55, wspace=0.30, top=0.85, bottom=0.1, left=0.08, right=0.92)

plt.savefig(out_dir+"LASSO_combined.tiff", dpi=300)

plt.show()
